In [6]:
import pandas as pd
import numpy as np
import numpy_financial as npf

class HomeModel:
    def __init__(self,
                 listvalue,  # total property value
                 interest_rate,  # per annum mortgage rate
                 discount_rate,  # general risk free rate
                 tax_proj_rate,  # general projection rate for tax liabilities
                 carry_proj_rate,  # general projection rate for carry income
                 property_proj_rate,  # general projection rate for property value
                 tax_rate,  # per annum tax rate
                 tax_pay_freq,  # tax payment frequency
                 tenor,  # tenor of the loan
                 down_percent,  # down payment percentage on the mortgage == initial equity in the property
                 down_pay_freq):  # frequency of down payment installations

        self.list_value = listvalue
        self.interest_rate = interest_rate
        self.discount_rate = discount_rate
        self.tax_rate = tax_rate
        self.tax_pay_freq = tax_pay_freq
        self.tax_projection_rate = tax_proj_rate
        self.carry_proj_rate = carry_proj_rate
        self.property_proj_rate = property_proj_rate
        self.tenor = tenor
        self.down_percent = down_percent
        self.down_pay_freq = down_pay_freq
        self.loan_amount = self.list_value * self.down_percent
        self.tax_df = self.get_cf_dataframe(self.generic_df_curve(self.discount_rate),
                                            self.generic_proj_curve(self.tax_projection_rate),
                                            self.vanilla_cf(frequency=self.tax_pay_freq,
                                                            period_cf=self.tax_rate * self.list_value * self.tax_pay_freq / 12))

        self.mortgage_payments = self.vanilla_cf(frequency=self.down_pay_freq,
                                                 period_cf=self.down_percent * self.list_value / (-12 * self.down_pay_freq * self.tenor))

        self.mortgage_schedule = self.get_mortgage_schedule()

    def generic_cf_dict(self, frequency=12):
        length = range(int(frequency / 12), self.tenor * frequency + 1)
        cf_library = {
            i: 0 for i in length
        }
        return cf_library

    def generic_df_curve(self, generic_discount_rate):
        dfcurve_ = self.generic_cf_dict()
        for period in dfcurve_:
            dfcurve_[period] = 1 / (1 + generic_discount_rate / 12) ** period
        return dfcurve_

    def generic_proj_curve(self, generic_projection_rate):
        projection_curve = self.generic_cf_dict()
        for period in projection_curve:
            projection_curve[period] = (1 + generic_projection_rate / 12) ** period
        return projection_curve

    def vanilla_cf(self, frequency=12, period_cf=0):
        vanilla_cf_ = self.generic_cf_dict()
        for period in vanilla_cf_.keys():
            if period % frequency == 0:
                vanilla_cf_[period] = period_cf
        return vanilla_cf_

    def calc_proj(self, cashflows, projection_curve):
        projected_cashflows = {
            cf_event: cashflows[cf_event] * projection_curve[cf_event] for cf_event in cashflows
        }
        return projected_cashflows

    def calc_disc(self, cashflows, discount_curve):
        discounted_cashflows = {
            cf_event: cashflows[cf_event] * discount_curve[cf_event] for cf_event in cashflows
        }
        return discounted_cashflows

    def get_cf_dataframe(self, discount_curve, projection_curve, cash_flows):
        df = pd.DataFrame(
            {
                'DF Curve': discount_curve,
                'Proj Curve': projection_curve,
                'Cashflows': cash_flows,
                'Fwd Cashflows': self.calc_proj(cash_flows, projection_curve),
                'Disc Fwd Cashflows': self.calc_disc(self.calc_proj(cash_flows, projection_curve), discount_curve)
            })
        return df

    def get_mortgage_schedule(self):
        df = pd.DataFrame(
            {
                'DF Curve': self.generic_df_curve(self.discount_rate),
                'Mortgage Payments': self.mortgage_payments,
            }
        )
        df['Accumulated Payments'] = df.index.values * df['Mortgage Payments']
        df['Remaining Loan'] = df['Accumulated Payments'] + self.loan_amount
        df['Interest Payments'] = df['Remaining Loan'] * self.interest_rate / 12
        df['Total Payments'] = df['Interest Payments'] + df['Mortgage Payments']
        return df


parameters = {
    'listvalue': 100000,
    'interest_rate': -0.05,
    'discount_rate': 0.016,
    'tax_rate': -0.05,
    'tax_pay_freq': 1,
    'tax_proj_rate': 0.025,
    'carry_proj_rate': 0.05,
    'property_proj_rate': 0.07,
    'tenor': 10,
    'down_percent': 0.5,
    'down_pay_freq': 1
}

NewProperty = HomeModel(**parameters)
NewProperty.tax_df

,DF Curve,Proj Curve,Cashflows,Fwd Cashflows,Disc Fwd Cashflows
1,0.998668,1.002083,-416.666667,-417.534722,-416.978751
2,0.997339,1.004171,-416.666667,-418.404586,-417.291068
3,0.996011,1.006263,-416.666667,-419.276262,-417.603620
4,0.994684,1.008359,-416.666667,-420.149755,-417.916405
5,0.993360,1.010460,-416.666667,-421.025067,-418.229425
6,0.992037,1.012565,-416.666667,-421.902202,-418.542680
7,0.990716,1.014675,-416.666667,-422.781165,-418.856169
8,0.989397,1.016789,-416.666667,-423.661959,-419.169893
9,0.988080,1.018907,-416.666667,-424.544588,-419.483851
10,0.986764,1.021030,-416.666667,-425.429056,-419.798045


In [9]:
NewProperty.mortgage_schedule

,DF Curve,Mortgage Payments,Accumulated Payments,Remaining Loan,Interest Payments,Total Payments
1,0.998668,-416.666667,-416.666667,49583.333333,-206.597222,-623.263889
2,0.997339,-416.666667,-833.333333,49166.666667,-204.861111,-621.527778
3,0.996011,-416.666667,-1250.000000,48750.000000,-203.125000,-619.791667
4,0.994684,-416.666667,-1666.666667,48333.333333,-201.388889,-618.055556
5,0.993360,-416.666667,-2083.333333,47916.666667,-199.652778,-616.319444
6,0.992037,-416.666667,-2500.000000,47500.000000,-197.916667,-614.583333
7,0.990716,-416.666667,-2916.666667,47083.333333,-196.180556,-612.847222
8,0.989397,-416.666667,-3333.333333,46666.666667,-194.444444,-611.111111
9,0.988080,-416.666667,-3750.000000,46250.000000,-192.708333,-609.375000
10,0.986764,-416.666667,-4166.666667,45833.333333,-190.972222,-607.638889
